# Databases

## Table of contents

1. [Engines and Connecting Strings](#Engines-and-Connecting-Strings)

## Engines 
[[go back to the top]](#Table-of-contents)

An engine requires information to connect with a database. This information is contained in a connection string such as:

    sqlite:///census.sqlite
    
In this string:
- `sqlite` is the database driver
- `census.sqlite` is a `SQLite` file in our local directory

`SQLAlchemy` is:
> the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

We will:
- Import `create_engine` from the `sqlalchemy` module.
- Create an engine using the `create_engine()` function that connects to a local `SQLite` file named `census.sqlite` using `sqlite` as driver
- Print the names of the tables it contains using the `.table_names()` method. 

In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///census.sqlite')

In [4]:
print('The database contains the following tables:\n')
print(engine.table_names())

The database contains the following tables:

['census', 'state_fact']


In [14]:
from sqlalchemy import inspect
inspector = inspect(engine)
print('Table names:', inspector.get_table_names())
# print('Column names:', inspector.get_columns(table_name))
for table_name in inspector.get_table_names():
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'])
        
        
for table_name in inspector.get_table_names():
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['type'])

Table names: ['census', 'state_fact']
Column: state
Column: sex
Column: age
Column: pop2000
Column: pop2008
Column: id
Column: name
Column: abbreviation
Column: country
Column: type
Column: sort
Column: status
Column: occupied
Column: notes
Column: fips_state
Column: assoc_press
Column: standard_federal_region
Column: census_region
Column: census_region_name
Column: census_division
Column: census_division_name
Column: circuit_court
Column: VARCHAR(30)
Column: VARCHAR(1)
Column: INTEGER
Column: INTEGER
Column: INTEGER
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)
Column: VARCHAR(256)


## Autoloading Tables from a Database
[[go back to the top]](#Table-of-contents)

We can use `SQLAlchemy` to automatically load tables from a database using `reflection`, the process of reading the database and building the metadata based on that information. For that we:
- Import the `Table` object from the `SQLAlchemy` package. 
- Use this `Table` object to read your table from the engine and autoload the columns.
- To autoload the columns with the engine, you have to specify the keyword arguments `autoload=True` and `autoload_with=engine` to `Table()`.

We will reflect the census table available on our engine into a variable called `census`. The metadata has already been loaded for you using `MetaData()` and is available in the variable `metadata`.

In [19]:
from sqlalchemy import MetaData
metadata = MetaData()

In [ ]:
metadata.reflect(engine)

In [21]:
metadata

MetaData(bind=None)

In [22]:
metadata.tables.values()

dict_values([Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None), Table('state_fact', MetaData(bind=None), Column('id', VARCHAR(length=256), table=<state_fact>), Column('name', VARCHAR(length=256), table=<state_fact>), Column('abbreviation', VARCHAR(length=256), table=<state_fact>), Column('country', VARCHAR(length=256), table=<state_fact>), Column('type', VARCHAR(length=256), table=<state_fact>), Column('sort', VARCHAR(length=256), table=<state_fact>), Column('status', VARCHAR(length=256), table=<state_fact>), Column('occupied', VARCHAR(length=256), table=<state_fact>), Column('notes', VARCHAR(length=256), table=<state_fact>), Column('fips_state', VARCHAR(length=256), table=<state_fact>), Column('assoc_press', VARCHAR(length=256), table=<state_fact>), Co

In [23]:
for table in metadata.tables.values():
    print(table.name)
    for column in table.c:
        print(column.name)

census
state
sex
age
pop2000
pop2008
state_fact
id
name
abbreviation
country
type
sort
status
occupied
notes
fips_state
assoc_press
standard_federal_region
census_region
census_region_name
census_division
census_division_name
circuit_court


- Import the `Table` object from `sqlalchemy`.
- Reflect the `census` table by using the `Table` object with the arguments
    - The name of the table as a string ('census').
    - The metadata, contained in the variable metadata.
    - autoload=True
    - The engine to autoload with - in this case, engine.
Print the details of census using the `repr()` function.

In [24]:
from sqlalchemy import Table

census = Table('census', metadata, autoload=True, autoload_with=engine)

# Print census table metadata
print(repr(census))

Table('census', MetaData(bind=None), Column('state', VARCHAR(length=30), table=<census>), Column('sex', VARCHAR(length=1), table=<census>), Column('age', INTEGER(), table=<census>), Column('pop2000', INTEGER(), table=<census>), Column('pop2008', INTEGER(), table=<census>), schema=None)


### Meaning of `Metadata`

MetaData is a container object that keeps together many different features of a database (or multiple databases) being described.


In [38]:
from sqlalchemy import *

metadata = MetaData()

To represent a table, use the `Table` class. Its two primary arguments are the table name, then the `MetaData` object which it will be associated with. The remaining positional arguments are mostly Column objects describing each column:

In [39]:
user = Table('user', metadata,
    Column('user_id', Integer, primary_key=True),
    Column('user_name', String(16), nullable=False),
    Column('email_address', String(60)),
    Column('password', String(20), nullable=False)
)

Above, a table called user is described, which contains four columns. The primary key of the table consists of the user_id column. Multiple columns may be assigned the primary_key=True flag which denotes a multi-column primary key, known as a composite primary key.

Note also that each column describes its datatype using objects corresponding to genericized types, such as Integer and String. SQLAlchemy features dozens of types of varying levels of specificity as well as the ability to create custom types. Documentation on the type system can be found at Column and Data Types.

### Accessing Tables and Columns
The MetaData object contains all of the schema constructs we’ve associated with it. It supports a few methods of accessing these table objects, such as the sorted_tables accessor which returns a list of each Table object in order of foreign key dependency (that is, each table is preceded by all tables which it references):

In [46]:
metadata.sorted_tables

[Table('user', MetaData(bind=None), Column('user_id', Integer(), table=<user>, primary_key=True, nullable=False), Column('user_name', String(length=16), table=<user>, nullable=False), Column('email_address', String(length=60), table=<user>), Column('password', String(length=20), table=<user>, nullable=False), schema=None)]

In [48]:
for t in metadata.sorted_tables:
    print(t.name)

user
